In [1]:
import sys
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math
import mlflow
import mlflow.pytorch
import dill
from uuid import uuid4
import seaborn as sns

torch.set_default_tensor_type(torch.DoubleTensor)

## Define bias and bias+UV decomposition models

In [2]:
class Bias_Model(nn.Module):
  
    def __init__(self, n_users, n_items, nan_map, rand_init=True, max_bias=3, min_bias=0, dim=None):
        super().__init__()
        torch.manual_seed(0)
        self.user_bias = nn.Parameter(torch.zeros(n_users))
        self.item_bias = nn.Parameter(torch.zeros(n_items))
        self.nan_map = nan_map
        if rand_init:
            nn.init.uniform_(self.user_bias, min_bias, max_bias)
            nn.init.uniform_(self.item_bias, min_bias, max_bias)
    
    def forward(self, X):
        y_pred = torch.cartesian_prod(self.user_bias, self.item_bias).sum(-1).view(len(self.user_bias), len(self.item_bias))
        y_pred = torch.where(self.nan_map, torch.zeros_like(y_pred), y_pred)
        return y_pred
        

In [3]:
class Bias_UV(nn.Module):
  
    def __init__(self, n_users, n_items, nan_map, rand_init=True, max_bias=3, min_bias=0, dim=15):
        super().__init__()
        torch.manual_seed(0)
        self.user_bias = nn.Parameter(torch.zeros(n_users))
        self.item_bias = nn.Parameter(torch.zeros(n_items))
        self.nan_map = nan_map
        self.U = nn.Parameter(torch.zeros(n_users, dim))
        self.V = nn.Parameter(torch.zeros(n_items, dim))
        if rand_init:
            nn.init.uniform_(self.user_bias, min_bias, max_bias)
            nn.init.uniform_(self.item_bias, min_bias, max_bias)
            nn.init.uniform_(self.U)
            nn.init.uniform_(self.V)
    
    def forward(self, X):
        y_pred_bias = torch.cartesian_prod(self.user_bias, self.item_bias).sum(-1).view(len(self.user_bias), len(self.item_bias))
        y_pred_UV = self.U @ self.V.t()
        y_pred = y_pred_bias + y_pred_UV
        y_pred = torch.where(self.nan_map, torch.zeros_like(y_pred), y_pred)
        return y_pred
        

## Defining metrics

In [4]:
def pointwise_mad_between_1darrays(arr1, arr2):
    """
    This function returns a np.array of (index, mad_value) tuple, 
    index signifying which index at both arrays were used to compute the value.
    If either of the arrays have NaN value at any given index, then its ignored for computation
    """
    try:
        if len(arr1) != len(arr2):
            raise ValueError("Array lengths should be equal")
        result = list()
        for i in range(len(arr1)):
            if ~np.isnan(arr1[i]) and ~np.isnan(arr2[i]):
                result.append((i, abs(float(arr1[i]) - float(arr2[i]))))
        return np.array(result)
    except Exception as e:
        error_type, error_instance, traceback = sys.exc_info()
        print(arr1, arr2)
        print(error_type, error_instance, traceback)
        

def pointwise_accuracy_ceil_or_floor(true_arr, pred_arr):
    """
    Returns a np.array of (index, accuracy_value) tuple.
    accuracy_val = 1, when true_arr E {ceil(pred_arr), floor(pred_arr)}, 0 otherwise. 
    Missing values in either arrays are discarded for computation
    """
    try:
        if len(true_arr) != len(pred_arr):
            raise ValueError("Array lengths should be equal")
        result = list()
        for i in range(len(true_arr)):
            if ~np.isnan(true_arr[i]) and ~np.isnan(pred_arr[i]):
                if int(true_arr[i]) in [math.floor(float(pred_arr[i])), math.ceil(float(pred_arr[i]))]:
                    result.append((i, 1))
                else:
                    result.append((i, 0))
        return np.array(result)
    except Exception as e:
        error_type, error_instance, traceback = sys.exc_info()
        print(true_arr, pred_arr)
        print(error_type, error_instance, traceback)
    
    
def pointwise_accuracy_ceil_or_floor_add1(true_arr, pred_arr):
    """
    Returns a np.array of (index, accuracy_value) tuple.
    accuracy_val = 1, when true_arr E {ceil(pred_arr) + 1, floor(pred_arr) - 1}, 0 otherwise. 
    Missing values in either arrays are discarded for computation
    """
    try:
        if len(true_arr) != len(pred_arr):
            raise ValueError("Array lengths should be equal")
        result = list()
        for i in range(len(true_arr)):
            if ~np.isnan(true_arr[i]) and ~np.isnan(pred_arr[i]):
                if int(true_arr[i]) in [
                    math.floor(float(pred_arr[i])) - 1 ,
                    math.ceil(float(pred_arr[i])) + 1,
                    math.floor(float(pred_arr[i])),
                    math.ceil(float(pred_arr[i]))
                ]:
                    result.append((i, 1))
                else:
                    result.append((i, 0))
        return np.array(result)
    except Exception as e:
        error_type, error_instance, traceback = sys.exc_info()
        print(true_arr, pred_arr)
        print(error_type, error_instance, traceback)

        
def metrics_collector(test_matrix, pred_matrix, common_indices, metrics='all'):
    metrics_map = {
        'accuracy_ceil_floor_add1': pointwise_accuracy_ceil_or_floor_add1,
        'accuracy_ceil_floor': pointwise_accuracy_ceil_or_floor,
        'mad': pointwise_mad_between_1darrays
    }
    try:
        if metrics!="all":
            raise ValueError("something something meri jaan")
        metrics_vals = {key:np.ones([1,3]) for key in metrics_map.keys()}
        for i in range(len(test_matrix)):
            for metric, func in metrics_map.items():
                _val = func(test_matrix[i], pred_matrix[common_indices[i][0]])
                _val = np.insert(_val, 0, i, axis=1)
                metrics_vals[metric] = np.append(metrics_vals[metric], _val, axis=0)
        return metrics_vals
    except Exception as e:
        error_type, error_instance, traceback = sys.exc_info()
        print(error_type, error_instance, traceback)

## Defining the training and testing functions

In [5]:
def train(epoch):
    
    model.train()
    
    opt.zero_grad()
    y_hat = model(X_train)
    loss = F.l1_loss(y_hat, X_train)
    loss.backward()
    opt.step()
    
    if epoch % args.log_interval == 0:
        mlflow.log_metric('sum_train_loss', loss.data.item()*10)
        mlflow.pytorch.log_model(pytorch_model=model, artifact_path="models/epoch{}".format(epoch))


def test(epoch, class_name):
    
    global _lowest_mad
    global _highest_acc
    
    model.eval()    
    with torch.no_grad():
    
        # create completed matrix
        if class_name == 'Bias_UV':
            bias_part = torch.cartesian_prod(model.user_bias, model.item_bias).sum(-1).view(len(model.user_bias), len(model.item_bias))
            uv_part = model.U @ model.V.t()
            completed_matrix = bias_part + uv_part
        else:
            completed_matrix = torch.cartesian_prod(model.user_bias, model.item_bias).sum(-1).view(len(model.user_bias), len(model.item_bias))
        
        # replace original training values using nan_mp
        completed_matrix = torch.where(nan_map, completed_matrix, train_matrix).numpy()
        
        # calculate metrics
        metrics = metrics_collector(test_matrix=test_matrix, pred_matrix=completed_matrix, common_indices=common_user_indices)
        
        # extract metrics
        accuracy_ceil_floor = np.delete(metrics['accuracy_ceil_floor'], 0, 0)[:,2]
        accuracy_ceil_floor_add1 = np.delete(metrics['accuracy_ceil_floor_add1'], 0, 0)[:,2]
        mad = np.delete(metrics['mad'], 0, 0)[:,2]
        
        # check if this epoch has recorded the best metrics
        if np.mean(mad) < _lowest_mad:
            _lowest_mad = np.mean(mad)
        if np.mean(accuracy_ceil_floor) > _highest_acc:
            _highest_acc = np.mean(accuracy_ceil_floor)
        
        # log metrics
        mlflow.log_metrics({
            'accuracy_ceil_floor': np.mean(accuracy_ceil_floor),
            'accuracy_ceil_floor_add1': np.mean(accuracy_ceil_floor_add1),
            'mean_abs_dev': np.mean(mad),
            'median_abs_dev': np.median(mad),
            'std_abs_dev': np.std(mad)
        })
        
        # log images in the last epoch
        if epoch == args.epochs:
            
            # log error plot
            image_dump_path = image_path + str(uuid4()) + ".png"
            error_df = error_plotter(metrics)
            sns.scatterplot(x='no_of_ratings_for_product', y='log(no.of_products_rated_by_user)', size='count', data=error_df)
            plt.savefig(image_dump_path, dpi=300)
            mlflow.log_artifact(image_dump_path, artifact_path="images")
            plt.close()
            
            # log user bias distribution
            image_dump_path = image_path + str(uuid4()) + ".png"
            sns.distplot(model.user_bias.detach().numpy().ravel()).set_title("User Bias distribution")
            plt.savefig(image_dump_path)
            mlflow.log_artifact(image_dump_path, artifact_path="images")  
            plt.close()

            
            # log user bias distribution
            image_dump_path = image_path + str(uuid4()) + ".png"
            sns.distplot(model.item_bias.detach().numpy().ravel()).set_title("Item Bias distribution")
            plt.savefig(image_dump_path)
            mlflow.log_artifact(image_dump_path, artifact_path="images")
            plt.close()

            
            # If its a UV model, log the U and V distribution as well
            if class_name == 'Bias_UV':
                
                # log user factor distribution
                image_dump_path = image_path + str(uuid4()) + ".png"
                sns.distplot(model.U.detach().numpy().ravel()).set_title("User factors distribution")
                plt.savefig(image_dump_path)
                mlflow.log_artifact(image_dump_path, artifact_path="images")
                plt.close()

                
                # log item factor distribution
                image_dump_path = image_path + str(uuid4()) + ".png"
                sns.distplot(model.V.detach().numpy().ravel()).set_title("Item factors distribution")
                plt.savefig(image_dump_path)
                mlflow.log_artifact(image_dump_path, artifact_path="images")
                plt.close()


## Loading the dataset and creating train and test matrices, get common indices

In [6]:
## Load the dataset
load_path = "file_server/processed_data/iteration1/magazine_subscription_subset_115X18241/"

# train_matrix
train_matrix = dill.load(open(load_path+"train_matrix.pkl", "rb"))

# train_users
train_users = dill.load(open(load_path+"train_users.pkl", "rb"))

# items (common for train and test)
items = dill.load(open(load_path+"items.pkl", "rb"))

# test_matrix
test_matrix = dill.load(open(load_path+"test_matrix.pkl", "rb"))

user_rating_counts = list(map(lambda x: np.count_nonzero(~np.isnan(train_matrix[x, :])), range(len(train_matrix))))
item_rating_counts = list(map(lambda x: np.count_nonzero(~np.isnan(train_matrix[:, x])), range(train_matrix.shape[1])))

# convert user ratings count to log space
user_rating_counts = np.log10(user_rating_counts)

# test_users
test_users = dill.load(open(load_path+"test_users.pkl", "rb"))


# Convert train_matrix to torch tensor and calc n_users and n_items
train_matrix = torch.from_numpy(train_matrix)

# Create nan_map, get n_users and n_items
nan_map = torch.isnan(train_matrix)
n_users, n_items = len(train_users), len(items)

# get index of users in train set who are present in test set
common_user_indices = np.argwhere(np.in1d(train_users, test_users))

## Set initial experiment params

In [7]:
class Params(object):
    def __init__(self, epochs, rand_init, log_interval):
        self.epochs = epochs
        self.rand_init = rand_init
        self.log_interval = log_interval

args = Params(epochs=50, rand_init=True, log_interval=5)

## Defining error plotter, creating user rating counts and item rating counts

In [8]:

# create a vectorizer to map to user and item ratings count from indices
user_count_mapper = np.vectorize(lambda x: user_rating_counts[int(x)])
item_count_mapper = np.vectorize(lambda x: item_rating_counts[int(x)])

# error plotter
def error_plotter(metrics):
    global user_count_mapper
    global item_count_mapper
    errors = metrics['accuracy_ceil_floor'][metrics['accuracy_ceil_floor'][:,2]==0]
    
    # apply mapper to errors
    errors[:,0] = user_count_mapper(errors[:,0])
    errors[:,1] = item_count_mapper(errors[:,1])

    # create a datframe for with item rating counts as 'x' and log(user rating counts) as 'y'
    error_df = pd.DataFrame(errors[:,:2], columns=['log(no.of_products_rated_by_user)','no_of_ratings_for_product'])
    error_df = pd.DataFrame({'count' : error_df.groupby( ['no_of_ratings_for_product', 'log(no.of_products_rated_by_user)'] ).size()}).reset_index()
    return error_df


## The learning loop over hyperparams

In [ ]:
# expt_id = mlflow.create_experiment('Fixed nH')

models = [Bias_UV, Bias_Model]
mlflow.set_experiment("with_l1_loss_plots")
image_path = "/home/bigdata/Desktop/projects/Rec_Sys/plots/"

for _model in models:
        
    if _model.__name__ == 'Bias_UV':
        dims = [5, 8, 10, 12, 15, 20]
    else:
        dims = [None]

    for lr in [0.4, 0.6, 0.9]:
        for weight_decay in [0, 1e-9, 1e-8, 1e-7, 1e-6]:
            for dim in dims:
                
                model = _model(n_users=n_users, n_items=n_items, nan_map=nan_map, dim=dim, rand_init=args.rand_init)
                opt = optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
                
                X_train = torch.where(nan_map, torch.zeros_like(train_matrix), train_matrix)

                with mlflow.start_run(run_name="lr={}|wd={}|dim={}".format(lr, weight_decay*1e10, dim)) as run:  
                    for key, value in vars(args).items():
                        mlflow.log_param(key, value)
                    mlflow.log_param('lr', lr)
                    mlflow.log_param('weight_decay', weight_decay)
                    mlflow.log_param('dimension', dim)
                    
                    _highest_acc = 0
                    _lowest_mad = float("inf")
                    
                    for epoch in range(1, args.epochs + 1):
                        train(epoch)
                        if epoch % args.log_interval == 0:
                            test(epoch, model.__class__.__name__)

                    mlflow.log_metrics({
                        'highest_accuracy': _highest_acc,
                        'lowest_mad': _lowest_mad
                    })
                    


2020/06/08 00:19:37 WARNING mlflow.tracking.context.git_context: Failed to import Git (the Git executable is probably not on your PATH), so Git SHA is not available. Error: Failed to initialize: Bad git executable.
The git executable must be specified in one of the following ways:
    - be included in your $PATH
    - be set via $GIT_PYTHON_GIT_EXECUTABLE
    - explicitly set via git.refresh()

All git commands will error until this is rectified.

This initial warning can be silenced or aggravated in the future by setting the
$GIT_PYTHON_REFRESH environment variable. Use one of the following values:
    - quiet|q|silence|s|none|n|0: for no warning or exception
    - warn|w|warning|1: for a printed warning
    - error|e|raise|r|2: for a raised exception

Example:
    export GIT_PYTHON_REFRESH=quiet

/home/bigdata/miniconda3/envs/rec_sys/lib/python3.7/site-packages/py4j/java_collections.py:13: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections

/home/bigdata/miniconda3/envs/rec_sys/lib/python3.7/site-packages/torch/serialization.py:402: UserWarning: Couldn't retrieve source code for container of type Bias_UV. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/bigdata/miniconda3/envs/rec_sys/lib/python3.7/site-packages/torch/serialization.py:402: UserWarning: Couldn't retrieve source code for container of type Bias_UV. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/bigdata/miniconda3/envs/rec_sys/lib/python3.7/site-packages/torch/serialization.py:402: UserWarning: Couldn't retrieve source code for container of type Bias_UV. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/bigdata/miniconda3/envs/rec_sys/lib/python3.7/site-packages/torch/serialization.py:402: UserWarning: Couldn't retrieve source code for container of type Bias_UV. It won't be checked fo

/home/bigdata/miniconda3/envs/rec_sys/lib/python3.7/site-packages/torch/serialization.py:402: UserWarning: Couldn't retrieve source code for container of type Bias_UV. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/bigdata/miniconda3/envs/rec_sys/lib/python3.7/site-packages/torch/serialization.py:402: UserWarning: Couldn't retrieve source code for container of type Bias_UV. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/bigdata/miniconda3/envs/rec_sys/lib/python3.7/site-packages/torch/serialization.py:402: UserWarning: Couldn't retrieve source code for container of type Bias_UV. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/home/bigdata/miniconda3/envs/rec_sys/lib/python3.7/site-packages/torch/serialization.py:402: UserWarning: Couldn't retrieve source code for container of type Bias_UV. It won't be checked fo